In [1]:
import pandas as pd
import numpy as np
import transformers
import json
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [95]:
train_data_path = "/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/training_data/train.json"
with open(train_data_path) as json_file:
    train_data = json.load(json_file)

In [7]:
train_df = pd.DataFrame()
uuid_list = []
hypothesis_list = []
section_list = []
type_list = []
primary_id_list = []
secondary_id_list = []
label_list = []
for uuid in train_data.keys():
    uuid_list.append(uuid)
    hypothesis_list.append(train_data[uuid]['Statement'])
    section_list.append(train_data[uuid]['Section_id'])
    type_list.append(train_data[uuid]['Type'])
    primary_id_list.append(train_data[uuid]['Primary_id'])
    try:
        secondary_id_list.append(train_data[uuid]['Secondary_id'])
    except:
        secondary_id_list.append(np.nan)
    label_list.append(train_data[uuid]['Label'])

train_df['uuid'] = uuid_list
train_df['type'] = type_list
train_df['section'] = section_list
train_df['primary_id'] = primary_id_list
train_df['secondary_id'] = secondary_id_list
train_df['hypothesis'] = hypothesis_list
train_df['label'] = label_list

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [13]:
import os
ctr_path = "/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/training_data/CT json/"
dir_list = os.listdir(ctr_path)
dir_list[0]

'NCT03329937.json'

In [21]:
report_path = ctr_path + dir_list[0]

with open(report_path) as json_file:
    report_data = json.load(json_file)

report_data['Intervention']

['INTERVENTION 1: ',
 '  Niraparib 200 mg',
 "  Participants received niraparib 200 milligrams (mg) orally once daily in 28-day treatment cycles. After 2 cycles, participants either underwent surgery, received additional cycles of niraparib (maximum of 6 cycles total), or received neoadjuvant chemotherapy, at physician's discretion. A breast magnetic resonance imaging (MRI) was performed at the end of cycle 2 and breast ultrasounds were performed at the end of each cycle, including any additional cycles beyond cycle 2. After completion of all neoadjuvant therapy participants proceeded to surgery, at which time pathological complete response (pCR) was assessed."]

In [33]:
def format_change(sentence):
  s = ""
  for sent in sentence:
    s += sent.strip() + ","
  return s

In [34]:
format_change(report_data['Intervention'])

"INTERVENTION 1:,Niraparib 200 mg,Participants received niraparib 200 milligrams (mg) orally once daily in 28-day treatment cycles. After 2 cycles, participants either underwent surgery, received additional cycles of niraparib (maximum of 6 cycles total), or received neoadjuvant chemotherapy, at physician's discretion. A breast magnetic resonance imaging (MRI) was performed at the end of cycle 2 and breast ultrasounds were performed at the end of each cycle, including any additional cycles beyond cycle 2. After completion of all neoadjuvant therapy participants proceeded to surgery, at which time pathological complete response (pCR) was assessed.,"

In [42]:
def report_generator(id1, id2, section):
  ctr_path = "/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/training_data/CT json/"
  if id2 == '_':
    id1_path = ctr_path + id1 + '.json'
    with open(id1_path) as json_file:
      id1_data = json.load(json_file)
    print(id1_data[section])
    sent = format_change(id1_data[section])
    print(sent)
  else:
    pass
  return sent

In [22]:
label_list = []
for label in train_df['label']:
    if label == 'Entailment':
        label_list.append(1)
    else:
        label_list.append(0)

train_df['label'] = label_list

train_df.head()

,uuid,type,section,primary_id,secondary_id,hypothesis,label
0,5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,0
1,86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",0
2,dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,1
3,20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,0
4,f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,0


In [43]:
ctr_sent_report_list = []

for i in range(train_df.shape[0]):
  type_ = train_df['type'].iloc[i]
  if type_.lower() == 'comparison':
    pass
  else:
    primary_id = train_df['primary_id'].iloc[i]
    report_sentence = report_generator(primary_id,'_',train_df['section'].iloc[i])
    print(report_sentence)
    ctr_sent_report_list.append(report_sentence)

  break

In [46]:
from torch.utils.data import Dataset

In [80]:
class CTRecordgenration:

  def __init__(self, data):
    self.data = data
    self.ctr_sent_report_list = []
    self.ctr_path = "/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/training_data/CT json/"

  def format_change(self, sentence):
    s = ""
    for sent in sentence:
      s += sent.strip() + ","
    return s

  def report_extractor(self, id1, id2, section):
    if id2 == '_':
      id1_path = self.ctr_path + id1 + '.json'
      with open(id1_path) as json_file:
        id1_data = json.load(json_file)
      sent = self.format_change(id1_data[section])
    else:
      id1_path = self.ctr_path + id1 + '.json'
      with open(id1_path) as json_file:
        id1_data = json.load(json_file)
      id2_path = self.ctr_path + id2 + '.json'
      with open(id2_path) as json_file:
        id2_data = json.load(json_file)
      id1_sent = self.format_change(id1_data[section])
      id2_sent = self.format_change(id2_data[section])
      sent = id1_sent + ' [SEP] ' + id2_sent
    return sent

  def ctr_list_generation(self):
    for i in range(self.data.shape[0]):
      type_ = self.data['type'].iloc[i]
      if type_.lower() == 'comparison':
        primary_id = self.data['primary_id'].iloc[i]
        secondary_id = self.data['secondary_id'].iloc[i]
        section = self.data['section'].iloc[i]
        self.ctr_sent_report_list.append(self.report_extractor(primary_id,secondary_id,section))
      else:
        primary_id = self.data['primary_id'].iloc[i]
        secondary_id = '_'
        section = self.data['section'].iloc[i]
        self.ctr_sent_report_list.append(self.report_extractor(primary_id,secondary_id,section))
    return self.ctr_sent_report_list

In [76]:
ctr_list = CTRecordgenration(train_df).ctr_list_generation()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [77]:
train_df['ctrs'] = ctr_list
train_df.head()

,uuid,type,section,primary_id,secondary_id,hypothesis,label,ctrs
0,5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,0,"INTERVENTION 1:,Diagnostic (FLT PET),Patients ..."
1,86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",0,"DISEASE CHARACTERISTICS:,Histologically or cyt..."
2,dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,1,"Adverse Events 1:,Total: 5/32 (15.63%),Febrile..."
3,20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,0,"Inclusion Criteria:,Patients with histologic c..."
4,f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,0,"INTERVENTION 1:,Laser Therapy Alone,therapist ..."


In [25]:
val_data_path = "/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/training_data/dev.json"
with open(val_data_path) as json_file:
    val_data = json.load(json_file)

In [28]:
val_df = pd.DataFrame()
uuid_list = []
hypothesis_list = []
section_list = []
type_list = []
primary_id_list = []
secondary_id_list = []
label_list = []
for uuid in val_data.keys():
    uuid_list.append(uuid)
    hypothesis_list.append(val_data[uuid]['Statement'])
    section_list.append(val_data[uuid]['Section_id'])
    type_list.append(val_data[uuid]['Type'])
    primary_id_list.append(val_data[uuid]['Primary_id'])
    try:
        secondary_id_list.append(val_data[uuid]['Secondary_id'])
    except:
        secondary_id_list.append(np.nan)
    label_list.append(val_data[uuid]['Label'])

val_df['uuid'] = uuid_list
val_df['type'] = type_list
val_df['section'] = section_list
val_df['primary_id'] = primary_id_list
val_df['secondary_id'] = secondary_id_list
val_df['hypothesis'] = hypothesis_list
val_df['label'] = label_list

In [30]:
label_list = []
for label in val_df['label']:
    if label == 'Entailment':
        label_list.append(1)
    else:
        label_list.append(0)

val_df['label'] = label_list

val_df.head()

,uuid,type,section,primary_id,secondary_id,hypothesis,label
0,1adc970c-d433-44d0-aa09-d3834986f7a2,Single,Results,NCT00066573,NaN,there is a 13.2% difference between the result...,0
1,6b9162d0-0816-46d4-81af-c60028dcc63b,Comparison,Eligibility,NCT00425854,NCT01224678,Patients with significantly elevated ejection ...,0
2,0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f,Comparison,Adverse Events,NCT02273973,NCT00281697,a significant number of the participants in th...,0
3,cc1f712a-2116-4e40-9810-f315e3fa5ff8,Single,Results,NCT00593346,NaN,the primary trial does not report the PFS or o...,1
4,904061c0-14fa-4f13-9118-9a41e24fa8eb,Single,Eligibility,NCT02340221,NaN,Prior treatment with fulvestrant or with a pho...,0


In [81]:
val_ctr_list = CTRecordgenration(val_df).ctr_list_generation()

val_df['ctrs'] = val_ctr_list
val_df.head()

,uuid,type,section,primary_id,secondary_id,hypothesis,label,ctrs
0,1adc970c-d433-44d0-aa09-d3834986f7a2,Single,Results,NCT00066573,NaN,there is a 13.2% difference between the result...,0,"Outcome Measurement:,Event-free Survival,Event..."
1,6b9162d0-0816-46d4-81af-c60028dcc63b,Comparison,Eligibility,NCT00425854,NCT01224678,Patients with significantly elevated ejection ...,0,"Inclusion criteria:,Inclusion Criteria:,Female..."
2,0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f,Comparison,Adverse Events,NCT02273973,NCT00281697,a significant number of the participants in th...,0,"Adverse Events 1:,Total: 20/167 (11.98%),Cardi..."
3,cc1f712a-2116-4e40-9810-f315e3fa5ff8,Single,Results,NCT00593346,NaN,the primary trial does not report the PFS or o...,1,"Outcome Measurement:,Local Control Using Ipsil..."
4,904061c0-14fa-4f13-9118-9a41e24fa8eb,Single,Eligibility,NCT02340221,NaN,Prior treatment with fulvestrant or with a pho...,0,"Inclusion Criteria:,Postmenopausal women with ..."


In [96]:
test_data_path = "/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/test.json"
with open(test_data_path) as json_file:
    test_data = json.load(json_file)

In [97]:
test_df = pd.DataFrame()
uuid_list = []
hypothesis_list = []
section_list = []
type_list = []
primary_id_list = []
secondary_id_list = []

for uuid in test_data.keys():
    uuid_list.append(uuid)
    hypothesis_list.append(test_data[uuid]['Statement'])
    section_list.append(test_data[uuid]['Section_id'])
    type_list.append(test_data[uuid]['Type'])
    primary_id_list.append(test_data[uuid]['Primary_id'])
    try:
        secondary_id_list.append(test_data[uuid]['Secondary_id'])
    except:
        secondary_id_list.append(np.nan)

test_df['uuid'] = uuid_list
test_df['type'] = type_list
test_df['section'] = section_list
test_df['primary_id'] = primary_id_list
test_df['secondary_id'] = secondary_id_list
test_df['hypothesis'] = hypothesis_list

In [98]:
test_ctr_list = CTRecordgenration(test_df).ctr_list_generation()

ValueError: Length of values (200) does not match length of index (5500)

In [99]:
test_df['ctrs'] = test_ctr_list
test_df.head()

,uuid,type,section,primary_id,secondary_id,hypothesis,ctrs
0,47468741-3624-4321-90ee-1c680a293b25,Comparison,Adverse Events,NCT00687102,NCT00819182,the record shows 5 total cases of asthenia and...,"Adverse Events 1:,Total: 0/733 (0.00%),Adverse..."
1,93e6ed2b-1e3f-4cbe-a49e-cbc74b31e1aa,Single,Eligibility,NCT01772004,NaN,'the primary clinical trial is open for partic...,Inclusion Criteria for dose escalation and exp...
2,ad9fe1aa-9798-4c53-ab23-23ac2924918c,Single,Intervention,NCT00118157,NaN,patients participating in the primary trial re...,"INTERVENTION 1:,Arm 1,Patients receive oral la..."
3,cdcb40f6-994c-4085-869a-dab87e67ba04,Single,Results,NCT01106898,NaN,"in the primary clinical trial, 97% of patients...","Outcome Measurement:,Recurrence-free Survival,..."
4,7df66c3f-de87-4542-9797-06325684db65,Comparison,Adverse Events,NCT02002533,NCT02734979,the primary trial has 4 more adverse events re...,"Adverse Events 1:,Total: 0/34 (0.00%), [SEP] A..."


In [100]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup

In [102]:
class Dataset(Dataset):

    def __init__(self, data):


        self.maxlen = 512

        self.labels = [label for label in data['label']]

        self.encoded_pair = [tokenizer(sent1, sent2,
                                      padding='max_length',
                                      truncation=True,
                                      max_length=self.maxlen,
                                      return_tensors='pt') for sent1, sent2 in zip(data['ctrs'],data['hypothesis'])]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):

        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):

        return self.encoded_pair[idx]

    def __getitem__(self, idx):


        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [84]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_output = self.sigmoid(linear_output).squeeze(dim=-1)

        return final_output

In [94]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device).view(-1).float()
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output.float(), train_label)
                total_loss_train += batch_loss.item()

                predictions = (output > 0.5).float()

                acc = (predictions == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device).view(-1).float()
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output.float(), val_label)
                    total_loss_val += batch_loss.item()

                    val_predictions = (output > 0.5).float()

                    acc = (val_predictions == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

EPOCHS = 2
model = BertClassifier()
LR = 1e-6

train(model, train_df, val_df, LR, EPOCHS)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epochs: 1 | Train Loss:  0.360                 | Train Accuracy:  0.484                 | Val Loss:  0.360                 | Val Accuracy:  0.480


100%|██████████| 850/850 [1:50:37<00:00,  7.81s/it]


Epochs: 2 | Train Loss:  0.353                 | Train Accuracy:  0.502                 | Val Loss:  0.364                 | Val Accuracy:  0.445


In [106]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    pred_label = []

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              pred_label.append(output.argmax(dim=-1))

    return pred_label

pred_label_list = evaluate(model, val_df)
print(pred_label_list)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

[tensor(1), tensor(1), tensor(0), tensor(1), tensor(0), tensor(0), tensor(1), tensor(0), tensor(1), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(0), tensor(1), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(0), tensor(1), tensor(0), tensor(1), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(0), tensor(1), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(1), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0)